# Simulate Inference Data

In [70]:
import pandas as pd
import numpy as np


df = pd.read_parquet('s3://traindata/train_raw.parquet', storage_options={"anon": False}).drop('class', axis='columns')

# quick and dirty way to add an event date column
df.insert(0, 'event_date', None)
num_days = 4
for chunk, date_ in zip(np.array_split(df, num_days), pd.date_range("2024-09-11", periods=num_days, freq="d")):
    chunk.event_date = date_
    df = pd.concat([df, chunk], axis='rows')
    
# we need to remove all original rows, those are duplicates that do not have a value in the event_date column
df = df.dropna()

df.head()

/usr/local/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/tmp/ipykernel_29224/2430470588.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, chunk], axis='rows')


,event_date,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season
0,2024-09-11,1372,2,2,10,3.807467,1545,11,1.804273
1,2024-09-11,1461,2,2,10,3.807467,1557,11,1.804273
2,2024-09-11,1371,2,2,10,3.612496,1566,11,1.804273
3,2024-09-11,1261,6,2,10,3.787572,1566,11,1.804273
4,2024-09-11,1305,6,2,10,3.711971,1464,11,0.943195


In [24]:
import s3fs

s3 = s3fs.S3FileSystem()
s3.mkdir("inferencedata")

In [71]:
df.to_parquet('s3://inferencedata/inference_raw.parquet', storage_options={"anon": False})

# Modell laden

In [65]:
import mlflow

# we never defined an official name, make sure you use the name of your own registered model here
model_name = "mushroom"

# this also only works if you set this alias for above model
model_version_alias = "champion"

model_uri = f"models:/{model_name}@{model_version_alias}"
model = mlflow.sklearn.load_model(model_uri)

# Trainingsdaten aus Object Store lesen

In [75]:
# yesterday = pd.Timestamp.today() - pd.Timedelta(days=1)
yesterday = pd.to_datetime('2024-09-13') - pd.Timedelta(days=1)

sel = [("event_date", "==", yesterday)]
df = pd.read_parquet('s3://inferencedata/inference_raw.parquet', filters=sel, storage_options={"anon": False})
df.head()

,event_date,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season
13509,2024-09-12,476,6,0,11,0.219270,495,7,1.804273
13510,2024-09-12,472,6,0,11,0.337788,493,7,0.027372
13511,2024-09-12,519,2,0,10,0.485011,597,7,1.804273
13512,2024-09-12,285,2,0,11,0.278104,523,7,1.804273
13513,2024-09-12,507,6,0,11,0.683960,532,7,0.943195


# Inferenz

In [76]:
df['prediction'] = model.predict(df.drop('event_date', axis='columns'))

In [77]:
df.head()

,event_date,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,prediction
13509,2024-09-12,476,6,0,11,0.219270,495,7,1.804273,0.0
13510,2024-09-12,472,6,0,11,0.337788,493,7,0.027372,0.0
13511,2024-09-12,519,2,0,10,0.485011,597,7,1.804273,0.0
13512,2024-09-12,285,2,0,11,0.278104,523,7,1.804273,0.0
13513,2024-09-12,507,6,0,11,0.683960,532,7,0.943195,1.0


In [78]:
df.prediction.value_counts()

prediction
0.0    7507
1.0    6002
Name: count, dtype: int64

# Zurückschreiben

...